In [1]:
import os
import numpy as np
from utils import model_utils

from signlens.params import *
from signlens.preprocessing.data import load_data_subset_csv, load_glossary
from signlens.preprocessing.preprocess import group_pad_sequences, encode_labels, decode_labels
from utils.model_utils import load_model

%load_ext autoreload

%autoreload 2


2024-03-23 14:26:52.691482: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-23 14:26:54.076962: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
test_data = load_data_subset_csv(balanced=True, csv_path=TRAIN_TEST_CSV_PATH)


Loading data subset from train_test.csv
    ℹ️ Filtered sequences with missing frames. Size reduced from 17233 to 17233 (100.00%)
    ℹ️ Filtered on n_frames = 100. Size reduced from 17233 to 17233 (100.00%)
    ℹ️ Filtered on n_classes = 10. Size reduced from 17233 to 690 (4.00%)
    ℹ️ Balanced data, with average of 34.5 elements per class. Size reduced from 690 to 345 (50.00%)
✅ Loaded 345 rows (2.00% of the original 17233 rows) from the dataset.


In [3]:
test_data

,path,participant_id,sequence_id,sign,n_frames,n_frames2,file_path
0,train_landmark_files_noface/62590/2461138136.p...,62590,2461138136,book,16,16,/home/bfrisque/code/benoitfrisque/signlens/raw...
1,train_landmark_files_noface/34503/3214793939.p...,34503,3214793939,book,43,43,/home/bfrisque/code/benoitfrisque/signlens/raw...
2,train_landmark_files_noface/22343/4214883602.p...,22343,4214883602,book,25,25,/home/bfrisque/code/benoitfrisque/signlens/raw...
3,train_landmark_files_noface/22343/1991418745.p...,22343,1991418745,book,8,8,/home/bfrisque/code/benoitfrisque/signlens/raw...
4,train_landmark_files_noface/2044/2844235998.pa...,2044,2844235998,book,6,6,/home/bfrisque/code/benoitfrisque/signlens/raw...
...,...,...,...,...,...,...,...
340,train_landmark_files_noface/25571/2727485578.p...,25571,2727485578,all,6,6,/home/bfrisque/code/benoitfrisque/signlens/raw...
341,train_landmark_files_noface/62590/632740077.pa...,62590,632740077,all,23,23,/home/bfrisque/code/benoitfrisque/signlens/raw...
342,train_landmark_files_noface/26734/3302718492.p...,26734,3302718492,all,28,28,/home/bfrisque/code/benoitfrisque/signlens/raw...
343,train_landmark_files_noface/37779/2479393513.p...,37779,2479393513,all,11,11,/home/bfrisque/code/benoitfrisque/signlens/raw...


# Check encoding

In [4]:
y = test_data.sign
y_encoded = encode_labels(y)
y_decoded = decode_labels(y_encoded)

In [5]:
y

0      book
1      book
2      book
3      book
4      book
       ... 
340     all
341     all
342     all
343     all
344     all
Name: sign, Length: 345, dtype: object

In [6]:
y_encoded

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [7]:
y_decoded

(['book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'book',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'drink',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
  'before',
 

# Model predict

In [8]:
model_name = "model 20240322-173411"
model, _ = load_model(model_name)

processed_data = group_pad_sequences(test_data.file_path.iloc[0:2])

prediction = model.predict([processed_data])

word, proba = decode_labels(prediction)

word, proba



Load latest model from local registry...

Load latest model from disk...


2024-03-23 14:26:57.613183: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 14:26:57.613717: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


✅ Model loaded from local disk /home/bfrisque/code/benoitfrisque/signlens/training_outputs/model 20240322-173411


/home/bfrisque/.pyenv/versions/3.10.6/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step


(['who', 'book'], array([0.24521649, 0.1531283 ], dtype=float32))